# espy - error simulations in python


In [1]:
print("We simulate erorrs in python!")

We simulate erorrs in python!


# Project

 - Simulate bit transmission on  physical layer
 - Implement and understand FEC and BEC algorithms and CRC
 - Compare algorithms and combinations under different channel models

# Goals

- Understanding for physical layer and bit error handling
- Learn how to do discrete event simulation in python
- Have fun with python

# Noise simulation  
### Creating random test data

In [2]:
import numpy
from espy.channelNoiseSimulator import channel_noise_simulator

#set seed
numpy.random.seed(100)

cns = channel_noise_simulator()
print(cns.create_random_bits_string(20))

00111100000100001001


# Noise simulation  
### Simulate simple errors

In [3]:
import numpy
from espy.channelNoiseSimulator import channel_noise_simulator
#seed
numpy.random.seed(100)
cns = channel_noise_simulator()

data = cns.create_random_bits_string(20)
noisy_data = cns.randomise_bits_string(data,0.1)

print(data)
print(noisy_data)
# highlight differences
print(''.join(str(e) for e in cns.compare_and_highlight_differences(data,noisy_data)))

00111100000100001001
00111100000100000000
Differences found: 2
00000000000000001001


# Noise simulation  
### Simulate burst errors

In [4]:
import numpy
from espy.channelNoiseSimulator import channel_noise_simulator
#seed
numpy.random.seed(100)
cns = channel_noise_simulator()

data = cns.create_random_bits_string(100)
noisy_data = cns.randomise_bits_burst_string(data,0.01,0.9,8)

print(data)
print(noisy_data)

# highlight differences
print(''.join(str(e) for e in cns.compare_and_highlight_differences(data,noisy_data)))

0011110000010000100101000111001001111100111111110111001101011100100111000000101010011000111100110010
0011110000010000100101000111001001111100111111110111001101011100100111000111010110011000111100110010
Differences found: 7
0000000000000000000000000000000000000000000000000000000000000000000000000111111100000000000000000000


# Cyclic Redundancy Check

In [31]:
from espy.crc import CyclicRedundancyCheck

crc3 = CyclicRedundancyCheck("1101")
cns = channel_noise_simulator()
msg = cns.create_random_bits_string(20)

msg_with_crc = crc3.crc_encode(msg)
msg_with_crc_noisy = cns.randomise_bits_string(msg_with_crc, 0.3)

print(msg)
print(msg_with_crc)
print(msg_with_crc_noisy)
print(''.join(str(e) for e in cns.compare_and_highlight_differences(msg_with_crc, msg_with_crc_noisy)))

01011001011010111001
01011001011010111001111
00111000001001101100111
Differences found: 9
01100001010011010101000


In [32]:
print(crc3.crc_check(msg_with_crc_noisy))

False


# Block Codes

In [7]:
from espy.block_code import *
import numpy as np

generator = np.array([[1, 0, 0, 0, 1, 1, 0],
                      [0, 1, 0, 0, 1, 0, 0],
                      [0, 0, 1, 0, 0, 1, 1],
                      [0, 0, 0, 1, 0, 0, 0]])

# init systematic block code with generator matrix
systematic_code = SystematicCode(generator)

# check encode
message = np.array([1, 0, 0, 1])
codeword = systematic_code.encode(message)
print(codeword == [1, 0, 0, 1, 1, 1, 1])

# check decode
decoded_message = systematic_code.decode(codeword)
print(decoded_message == message)



[ True  True  True  True  True  True False]
[ True False False False]


In [8]:
from espy.block_code import *
import numpy as np

# init systematic block code with generator matrix
systematic_code = SystematicCode(generator)

# check code word table
codeword_table = systematic_code.codeword_table()
print(codeword_table == [[0, 0, 0, 0, 0, 0, 0],
                          [1, 0, 0, 0, 1, 1, 0],
                          [0, 1, 0, 0, 1, 0, 0],
                          [1, 1, 0, 0, 0, 1, 0],
                          [0, 0, 1, 0, 0, 1, 1],
                          [1, 0, 1, 0, 1, 0, 1],
                          [0, 1, 1, 0, 1, 1, 1],
                          [1, 1, 1, 0, 0, 0, 1],
                          [0, 0, 0, 1, 0, 0, 0],
                          [1, 0, 0, 1, 1, 1, 0],
                          [0, 1, 0, 1, 1, 0, 0],
                          [1, 1, 0, 1, 0, 1, 0],
                          [0, 0, 1, 1, 0, 1, 1],
                          [1, 0, 1, 1, 1, 0, 1],
                          [0, 1, 1, 1, 1, 1, 1],
                          [1, 1, 1, 1, 0, 0, 1]])

# check validate
print(systematic_code.validate(codeword))
print(not systematic_code.validate([1, 1, 1, 1, 0, 0, 0]))

[[ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True]]
True
True


# ARQ Protocol Simulation

In [9]:
import simpy, logging
from espy.simulation.client import Client
from espy.simulation.router import Router

NUM_CLIENTS = 3
SIM_TOTAL_DURATION = 1000
ROUTER_DROP_PROB = 0.3
SPEED = 0.2
LOG_LEVEL = logging.INFO

def init_clients():
    clients = []
    for i in range(NUM_CLIENTS):
        clients.append(Client(env, i, router, NUM_CLIENTS))
    return clients

if __name__ == "__main__":
    logging.basicConfig(format='[%(levelname)-7s] %(name)8s: %(message)s', level=LOG_LEVEL)

    # init simpy and network
    env = simpy.Environment()
    router = Router(env, ROUTER_DROP_PROB, SPEED)
    clients = init_clients()
    router.store_clients(clients)

    logging.info("Simulation starting")
    env.run(until=SIM_TOTAL_DURATION)
    logging.info("Simulation finished")

[INFO   ]     root: Simulation starting
[INFO   ] Client 0: Send message to 2: Hello World!
[INFO   ] Client 2: Received message from 0: Hello World!
[INFO   ] Client 2: Send message to 0: ACK
[INFO   ] Client 0: Received message from 2: ACK
[INFO   ] Client 1: Send message to 0: Hello World!
[INFO   ] Client 0: Received message from 1: Hello World!
[INFO   ] Client 0: Send message to 1: ACK
[INFO   ] Client 1: Received message from 0: ACK
[INFO   ] Client 2: Send message to 0: Hello World!
[INFO   ] Client 0: Received message from 2: Hello World!
[INFO   ] Client 0: Send message to 2: ACK
[INFO   ] Client 2: Received message from 0: ACK
[INFO   ]     root: Simulation finished
